# Segmenting and Clustering Neighborhoods in Toronto


## Web scrapping of wikipedia " List of postal codes of Canada" page

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Web scrapping of wikipedia ( List of postal codes of Canada Toronto ) page</a>

2. <a href="#item2">Get the latitude and longitude of the Neighborhoods</a>

3. <a href="#item2">Explore Neighborhoods in Toronto City</a>

4. <a href="#item3">Analyze Each Neighborhood</a>

5. <a href="#item4">Cluster Neighborhoods</a>

6. <a href="#item5">Examine Clusters</a>    
</font>
</div>

# 1- Web Scrapping of Wikipedia

import of the pakages.<br>
I  use beutifullsoup to do the webscrapping (i assume that only the first html table conatins the postal code)

In [222]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

headers = requests.utils.default_headers()

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

#print(soup.prettify())

tabs =soup.find_all("table")# i only take the first table in the page
tab=tabs[0]

Read the rows and fill a dictionnary (the key is the postal code)

In [97]:
dic={}#i create a dictionnary from the html table, the key is the postal code

#loops on the tr and td html tags
for row in tab.find_all('tr'):
   
    columns = row.find_all('td')
   
    postalCode=""
    borough=""
    neighborhood=""

    tds=[0,0,0]
    ind=0

    for column in columns:
       tds[ind]= column.get_text().rstrip()#\n suppress
       ind+=1
    
    #i skip the header row
    if ind==0:
        continue  
  
    #i read the values 
    postalCode=tds[0]
    borough=tds[1]

    if(tds[2]!='Not assigned'):
        neighborhood=tds[2]
    else:
        neighborhood=borough

    #filling the dictionnary
    if postalCode in dic :
        #the postalcode already exist in the dictionnary
        dic[postalCode]=dic[postalCode][1]+','+neighborhood
    else:
        if(borough!='Not assigned'):
            dic[postalCode]=[borough,neighborhood]
        


In [66]:
#building the dataframe
df=pd.DataFrame.from_dict(dic,columns=['Borough','Neighborhood'],  orient='index')
df['PostalCode']=df.index
df.reset_index(drop=True,inplace=True)
df = df.reindex(columns = ['PostalCode', 'Borough','Neighborhood'])
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [67]:
df.shape

(103, 3)

#  2-Get the latitude and longitude of the Neighborhoods

I ve tried to use geocoder but it doesn't work for me.
I've used geopy / normatin (open street map) it works better but some postal codes doesn't return any result, so i've decided to use the csv file containing the longitude and latitue.
I let the code below but idon't use it in this assignmet.

In [98]:
from geopy.geocoders import Nominatim

# function that return latitude and longitude from postalCode,borough,neighborhood
def getLocation(postalCode,borough,neighborhood):
    geolocator = Nominatim(user_agent="Coursra_Toronto")
    str0 ='{},{},Toronto, Ontario,{}'.format(neighborhood,borough,postalCode) 
    str1='{},Toronto, Ontario'.format(postalCode)
    str2='{},{},canada'.format(neighborhood,borough)

    location = geolocator.geocode(str1)

    #print(postalCode)
    print(str0)
    #print(str2)
    if location!=None:
        print(location.address,(location.latitude, location.longitude))
        #print((location.latitude, location.longitude))
        #print(location.raw)
    else:
        location = geolocator.geocode(str2)
        if location!=None:
            print(location.address,(location.latitude, location.longitude))
        else:
            print("No result!")

i loop over datas (postal code etc) to find gps coordinate
some results are missing so i will use the provided csv file

In [99]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="Coursra_Toronto")

for line in df.itertuples():
    #print(line.Borough)
    #print(line.PostalCode,line.Borough,line.Neighborhood)
    getLocation(line.PostalCode,line.Borough,line.Neighborhood)

Parkwoods,North York,Toronto, Ontario,M3A
Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada (43.6534817, -79.3839347)
Victoria Village,North York,Toronto, Ontario,M4A
Victoria Village, Don Valley East, North York, Toronto, Golden Horseshoe, Ontario, M4A 2B1, Canada (43.732658, -79.3111892)
Regent Park, Harbourfront,Downtown Toronto,Toronto, Ontario,M5A
Pas de resultat!
Lawrence Manor, Lawrence Heights,North York,Toronto, Ontario,M6A
Pas de resultat!
Queen's Park, Ontario Provincial Government,Downtown Toronto,Toronto, Ontario,M7A
Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada (43.6534817, -79.3839347)
Islington Avenue, Humber Valley Village,Etobicoke,Toronto, Ontario,M9A
Islington Avenue, Humber Valley, Etobicoke, Etobicoke Centre, Etobicoke, Toronto, Golden Horseshoe, Ontario, M9A 3L8, Canada (43.6684992, -79.5339701)
Malvern, Rouge,Scarborough,Toronto, Ontario,M1B
Toronto, Golden Horseshoe, Ontario, M5H 2N2, Canada (43.6534817, -79.3839347)
Don Mills,North York,Toronto, Ontari

finally i use the csv file


In [75]:
dfCoord = pd.read_csv('Geospatial_Coordinates.csv')
dfCoord.columns=['PostalCode','Latitude','Longitude']
dfCoord.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


I merge the two dataframes using the postal code

In [106]:
df1 = pd.merge(df, dfCoord, on='PostalCode')
df1.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


import geocoder
#g = geocoder.canadapost('M6A, Toronto')
g = geocoder.canadapost('M6A,Toronto,Ontario')
print(g.postal)
print(g.address, g.country, g.latlng)

In [109]:
df1.to_csv('Toronto_Borough_Coord.csv')

In [133]:
#i filter the boroughs to kepp onmy those containing "Toronto"
toronto_boroughs = df1[df1['Borough'].str.contains("Toronto", case=False)]
toronto_boroughs.shape
#boroughs.head()

(39, 5)

In [134]:
# create map of Toronto using latitude and longitude values

import folium

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Coursra_Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Borough'], toronto_boroughs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [138]:
CLIENT_ID = 'WYCBJKQTTOGLP2DE3JZVMOEEHQX2LK0KLSZWADASUDKBM3TL' # your Foursquare ID
CLIENT_SECRET = 'LUXIEPZ1XOFLT4F3RQVCGY3WRP3VXGDLAFUYJ14WEUVJMUUE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [139]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [140]:
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                   latitudes=toronto_boroughs['Latitude'],
                                   longitudes=toronto_boroughs['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [141]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [146]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",17,17,17,17,17,17
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,64,64,64,64,64,64
Christie,17,17,17,17,17,17
Church and Wellesley,77,77,77,77,77,77
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,36,36,36,36,36,36


In [145]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


In [168]:
 toronto_venues['Neighborhood'].value_counts()

Harbourfront East, Union Station, Toronto Islands                                                             100
Toronto Dominion Centre, Design Exchange                                                                      100
Garden District, Ryerson                                                                                      100
First Canadian Place, Underground city                                                                        100
Commerce Court, Victoria Hotel                                                                                100
Richmond, Adelaide, King                                                                                       92
Stn A PO Boxes                                                                                                 92
St. James Town                                                                                                 81
Church and Wellesley                                                                    

In [192]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.insert(0, 'Neighborhood_', toronto_venues['Neighborhood'] )
#one venue category is Neighborhood so i rename the column title
toronto_onehot.head()

,Neighborhood_,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [215]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_').mean().reset_index()

In [216]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [217]:
import numpy as np # library to handle data in a vectorized manner
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood_']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood_'] = toronto_grouped['Neighborhood_']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood_,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Café,Cheese Shop,Seafood Restaurant,Breakfast Spot,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Stadium,Italian Restaurant,Intersection,Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Brewery,Restaurant,Smoke Shop,Fast Food Restaurant,Auto Workshop,Farmers Market,Burrito Place,Spa,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Harbor / Marina,Sculpture Garden,Coffee Shop,Plane,Boat or Ferry,Bar,Airport Terminal,Airport Gate
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Thai Restaurant,Ice Cream Shop,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint


In [218]:

# import k-means from clustering stage
from sklearn.cluster import KMeans


# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 1, 0, 2, 0,
       0, 0, 0, 0, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [219]:
toronto_boroughs.columns


Index(['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [220]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_boroughs
toronto_merged.columns =['PostalCode', 'Borough', 'Neighborhood_', 'Latitude', 'Longitude']


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood_'), on='Neighborhood_')

#toronto_merged = pd.merge(toronto_merged, neighborhoods_venues_sorted, left_on='Neighborhood', right_on='Neighborhood_')
toronto_merged.head()


,PostalCode,Borough,Neighborhood_,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Shoe Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,College Auditorium
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Tea Room
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Clothing Store,Gym,Moroccan Restaurant,Creperie
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store


In [223]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood_'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [229]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood_,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,"Regent Park, Harbourfront",0,Coffee Shop,Bakery,Pub,Park,Theater,Breakfast Spot,Café,Restaurant,Mexican Restaurant,Shoe Store
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,Park,College Auditorium
9,Downtown Toronto,"Garden District, Ryerson",0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Restaurant,Italian Restaurant,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Tea Room
15,Downtown Toronto,St. James Town,0,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Restaurant,Clothing Store,Gym,Moroccan Restaurant,Creperie
19,East Toronto,The Beaches,0,Trail,Neighborhood,Pub,Health Food Store,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store
20,Downtown Toronto,Berczy Park,0,Coffee Shop,Cocktail Bar,Bakery,Beer Bar,Restaurant,Café,Cheese Shop,Seafood Restaurant,Breakfast Spot,Irish Pub
24,Downtown Toronto,Central Bay Street,0,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Thai Restaurant,Ice Cream Shop,Japanese Restaurant,Bubble Tea Shop,Salad Place,Burger Joint
25,Downtown Toronto,Christie,0,Grocery Store,Café,Park,Restaurant,Nightclub,Baby Store,Athletics & Sports,Italian Restaurant,Diner,Candy Store
30,Downtown Toronto,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Deli / Bodega,Thai Restaurant,Gym,Hotel,Clothing Store,Salad Place,Steakhouse
31,West Toronto,"Dufferin, Dovercourt Village",0,Bakery,Pharmacy,Middle Eastern Restaurant,Bank,Bar,Café,Pool,Supermarket,Music Venue,Grocery Store


In [230]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood_,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,Lawrence Park,1,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [231]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood_,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,"Moore Park, Summerhill East",2,Park,Playground,Summer Camp,Restaurant,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
91,Downtown Toronto,Rosedale,2,Park,Trail,Playground,Yoga Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run


In [232]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood_,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",3,Jewelry Store,Mexican Restaurant,Trail,Sushi Restaurant,Yoga Studio,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [233]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1,2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood_,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,Roselawn,4,Pool,Garden,Yoga Studio,Cupcake Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
